In [ ]:
U_electrostatics;
U_electrostatics = switch * c_RF * u_RF;
switch = step(-t) + step(t) * step(1-t) * (1 + (t^3) * (-10 + t * (15 - (6 * t))));
t = (r_eff - r_switch) / (r_cutoff - r_switch);
c_RF = 138.935456 * chargeProd;
chargeProd = charge1 * charge2;
charge1 = charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 ) + charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ) * ( nonscale_lambda_electrostatics * nonscale_region1 );
charge2 = charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 ) + charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ) * ( nonscale_lambda_electrostatics * nonscale_region2 );
# charge1 = select(1 - both_environment, charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 ) + charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ), charge_old1) * ( nonscale_lambda_electrostatics * nonscale_region1 );
# charge2 = select(1 - both_environment, charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 ) + charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ), charge_old2) * ( nonscale_lambda_electrostatics * nonscale_region2 );
# both_environment = environment_region1 * environment_region2;
u_RF = 1 / r_eff;
r_eff = sqrt(r^2 + w^2);
w = step(unique_new1 + unique_new2 + unique_old1 + unique_old2 - 0.1) * r_cutoff * (1. - ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 ) - ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 )) * 1.0;
r_switch = r_cutoff * 0.99;
eps_RF = 78.8;
r_cutoff = 1.4;
w_scale = 1.0

# core1, core2 is never used
# in t, r_cutoff - r_switch is 0 -- made the switching ratio for testing 0.99
# w -- offset term that prevents singularities (lifts it into the 4th dimension), similar to softcore
# w -- is this correct, should we be using environment_region2 and alchemical_region02 anywhere?

In [11]:
U_sterics; 
U_sterics = 4 * epsilon * x * (x - 1.0);
x = (sigma / r_eff)^6;
sigma = (sigma1 + sigma2) / 2;
sigma1 = select(1 - both_environment, sigma_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 ) + sigma_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 ), sigma_old1);
sigma2 = select(1 - both_environment, sigma_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 ) + sigma_new2 * ( 1 * environment_region2+lambda_0_sterics_new * alchemical_region_02 ), sigma_old2);
epsilon = sqrt(epsilon1 * epsilon2);
epsilon1 = select(1 - both_environment, epsilon_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 ) + epsilon_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 ), epsilon_old1) * ( nonscale_lambda_sterics * nonscale_region1 );
epsilon2 = select(1 - both_environment, epsilon_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 ) + epsilon_new2 * ( 1 * environment_region2+lambda_0_sterics_new * alchemical_region_02 ), epsilon_old2) * ( nonscale_lambda_sterics * nonscale_region2 );
both_environment = environment_region1 * environment_region2;
r_eff = sqrt(r^2 + w^2);
w = step(unique_new1 + unique_new2 + unique_old1 + unique_old2 - 0.1) * r_cutoff * (1. - ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 ) - ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 )) * 1.0;
r_cutoff = 1.4;
w_scale = 1.0;

# steric switching ratio is not used here

'U_sterics; U_sterics = 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff)^6; sigma = (sigma1 + sigma2) / 2; sigma1 = select(1 - both_environment, sigma_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 ) + sigma_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 ), sigma_old1); sigma2 = select(1 - both_environment, sigma_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 ) + sigma_new2 * ( 1 * environment_region2+lambda_0_sterics_new * alchemical_region_02 ), sigma_old2); epsilon = sqrt(epsilon1 * epsilon2); epsilon1 = select(1 - both_environment, epsilon_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 ) + epsilon_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 ), epsilon_old1) * ( nonscale_lambda_sterics * nonscale_region1 ); epsilon2 = select(1 - both_environment, epsilon_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 ) 

Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In [2]:
def test_nb_energies(htf, is_electrostatics=True, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)

    # Get system and old/new positions. Remove barostat
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)

    # Get nb force
    nb_force_index = 3
    nb_force = system.getForce(nb_force_index) 

    # Zero the sterics if we're checking electrostatics energies (and vis versa)
    if is_electrostatics:
        for i in range(nb_force.getNumParticles()):
            charge, sigma, epsilon = nb_force.getParticleParameters(i)
            nb_force.setParticleParameters(i, charge, sigma, epsilon*0)

        for i in range(nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
            nb_force.setExceptionParameters(i, p1, p2, chargeProd, sigma, epsilon*0)
        
#         print("zeroing elec too")
#         for i in range(nb_force.getNumParticles()):
#             charge, sigma, epsilon = nb_force.getParticleParameters(i)
# #             if i in list(htf._atom_classes['environment_atoms']) +  list(htf._alchemical_regions_by_type['core_atoms']):
#             nb_force.setParticleParameters(i, charge*0, sigma, epsilon)
        
#         print("zeroing the elec exceptions too")
#         for i in range(nb_force.getNumExceptions()):
#             p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
#             if p1 in htf._atom_classes['environment_atoms'] or p2 in htf._atom_classes['environment_atoms'] or p1 in htf._atom_classes['core_atoms'] or  p2 in htf._atom_classes['core_atoms']:
#                 print(f"zeroing {p1} {p2}")
#                 nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon)
    else:
        for i in range(nb_force.getNumParticles()):
            charge, sigma, epsilon = nb_force.getParticleParameters(i)
            nb_force.setParticleParameters(i, charge*0, sigma, epsilon)

        for i in range(nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
            nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon)
        
#         print("zeroing steric exceptions too")
#         for i in range(nb_force.getNumExceptions()):
#             p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
#             nb_force.setExceptionParameters(i, p1, p2, chargeProd, sigma, epsilon*0)
        
#         print("zeroing sterics too")
#         for i in range(nb_force.getNumParticles()):
#             charge, sigma, epsilon = nb_force.getParticleParameters(i)
#             nb_force.setParticleParameters(i, charge, sigma, epsilon*0)
        
    # Get system and positions. Remove barostat
    hybrid_system = htf.hybrid_system
    hybrid_positions = htf.hybrid_positions

    # Get custom nb force 
    if is_electrostatics:
        custom_nb_force_index = 3 
        custom_nb_exceptions_force_index = 5 
    else:
        custom_nb_force_index = 4
        custom_nb_exceptions_force_index = 6
    custom_nb_force = hybrid_system.getForce(custom_nb_force_index)
    custom_nb_exceptions_force = hybrid_system.getForce(custom_nb_exceptions_force_index)


#     for i in range(custom_nb_force.getNumParticles()):
#         params = custom_nb_force.getParticleParameters(i)
#         if i in list(htf._atom_classes['environment_atoms'])  +  list(htf._alchemical_regions_by_type['core_atoms']):
#             params = list(params)
#             params[6] = params[6]*0
#             params[7] = params[7]*0
#             custom_nb_force.setParticleParameters(i, params)
            
#     for i in range(custom_nb_exceptions_force.getNumBonds()):
#         p1, p2, params = custom_nb_exceptions_force.getBondParameters(i)
# #         print(p1, p2, params)
#         if p1 in htf._atom_classes['environment_atoms'] or p2 in htf._atom_classes['environment_atoms'] or p1 in htf._atom_classes['core_atoms'] or p2 in htf._atom_classes['core_atoms']:
#             print(f"zeroing in custom {p1} {p2}")
#             params = list(params)
#             params[6] = params[6]*0
#             params[7] = params[7]*0
#             custom_nb_exceptions_force.setBondParameters(i, p1, p2, params)
        
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    if is_electrostatics:
        lambda_old_str = 'lambda_0_electrostatics_old'
        lambda_new_str = 'lambda_0_electrostatics_new'
        lambda_old_exceptions_str = 'lambda_0_electrostatics_exceptions_old'
        lambda_new_exceptions_str = 'lambda_0_electrostatics_exceptions_new'
    else:
        lambda_old_str = 'lambda_0_sterics_old'
        lambda_new_str = 'lambda_0_sterics_new'
        lambda_old_exceptions_str = 'lambda_0_sterics_exceptions_old'
        lambda_new_exceptions_str = 'lambda_0_sterics_exceptions_new'
    
    for i in range(custom_nb_force.getNumGlobalParameters()):
        if custom_nb_force.getGlobalParameterName(i) == lambda_old_str:
            custom_nb_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_nb_force.getGlobalParameterName(i) == lambda_new_str:
            custom_nb_force.setGlobalParameterDefaultValue(i, lambda_new)

    for i in range(custom_nb_exceptions_force.getNumGlobalParameters()):
        if custom_nb_exceptions_force.getGlobalParameterName(i) == lambda_old_exceptions_str:
            custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_nb_exceptions_force.getGlobalParameterName(i) == lambda_new_exceptions_str:
            custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Get energy components of standard nb force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    # Get energy components of custom nb force
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)

    assert np.isclose([components_other[nb_force_index][1]], np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [custom_nb_force_index, custom_nb_exceptions_force_index] ])), f"standard nonbonded: {components_other[nb_force_index][1]}, custom nonbonded/bond: {np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [custom_nb_force_index, custom_nb_exceptions_force_index] ])}"

    type_str = 'electrostatics' if is_electrostatics else 'sterics'
    print(f"Success! Custom nb force and standard nb force {type_str} energies are equal!")

    if check_scale:
        
        ## Get custom nb force and hybrid positions
        hybrid_system = htf_copy.hybrid_system
        custom_force = hybrid_system.getForce(custom_nb_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        ## Get energy components of custom torsion force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_force.getNumGlobalParameters()):
            if custom_force.getGlobalParameterName(i) == 'scale_lambda_0_electrostatics_exceptions':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_force.getGlobalParameterName(i) == 'interscale_lambda_0_electrostatics_exceptions':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_force.getGlobalParameterName(i) == 'scale_lambda_0_sterics_exceptions':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_force.getGlobalParameterName(i) == 'interscale_lambda_0_sterics_exceptions':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_force.getGlobalParameterName(i) == 'scale_lambda_0_electrostatics':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_force.getGlobalParameterName(i) == 'scale_lambda_0_sterics':
                custom_force.setGlobalParameterDefaultValue(i, 0.5)
            
        ## Get energy components of custom torsion force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[custom_nb_force_index][1]], [components_hybrid_scaled[custom_nb_force_index][1]])
        
        print(f"Success! Scaling the custom nb {type_str} force changes the energy")
        
    return context_hybrid

# Alanine dipeptide in vacuum

In [29]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
atp.system.getForce(3).setUseDispersionCorrection(False)
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.7099781933273569.
INFO:geometry:	reduced angle potential = 0.009251960418866147.
INFO:geometry:	reduced angle potential = 0.0015548695789022475.
INFO:geometry:	reduced angle potential = 0.09331614762646692.
INFO:geometry:	reduced angle potential = 0.002544618530922158.
INFO:geometry:	reduced angle potential = 0.009128386984563263.
INFO:geometry:	reduced angle potential = 0.12900683791035714.
INFO:geometry:	reduced angle potential = 0.6031006409388495.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 a

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -27051.04994557314


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda'], ['nonscale_region1'], ['nonscale_region2']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quan

In [30]:
# Save htf
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)
    

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [31]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [32]:
htf._topology_proposal.old_system.getForce(3).getUseDispersionCorrection()

False

In [33]:
htf.hybrid_system.getForce(3).getUseLongRangeCorrection()

False

In [34]:
test_nb_energies(htf)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -52.66819022395389), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1809935780184042), ('CustomAngleForce', 6.79008469282173), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [35]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [36]:
test_nb_energies(htf, is_electrostatics=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', 13.218262478353564), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1809935780184042), ('CustomAngleForce', 6.79008469282173), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_nb_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.180469882789552), ('HarmonicAngleForce', 6.335937267696079), ('PeriodicTorsionForce', 26.03839300176716), ('NonbondedForce', -117.63771851625125), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1809935780184042), ('CustomAngleForce', 6.79008469282173), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -89.483737175813), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', -28.153986933079743), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [39]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [40]:
test_nb_energies(htf, is_electrostatics=False, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.180469882789552), ('HarmonicAngleForce', 6.335937267696079), ('PeriodicTorsionForce', 26.03839300176716), ('NonbondedForce', 86.18239438638857), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1809935780184042), ('CustomAngleForce', 6.79008469282173), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 70.47845182209416), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 15.70392700440809), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


### Test one alchemical region, one scale region

In [14]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 1.1782244583437067.
INFO:geometry:	reduced angle potential = 0.34552700837882344.
INFO:geometry:	reduced angle potential = 0.2832082868608191.
INFO:geometry:	reduced angle potential = 0.4196004176498398.
INFO:geometry:	reduced angle potential = 1.1278924582914134.
INFO:geometry:	reduced angle potential = 0.32070858271014435.
INFO:geometry:	reduced angle potential = 0.5288541638669997.
INFO:geometry:	reduced angle potential = 0.09520287230670622.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle for

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26941.89920790118


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda', 'scale_lambda_0'], ['nonscale_region1', 'scale_region_01'], ['nonscale_region2', 'scale_region_02']]
INFO:relative:No MonteCarloBa

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;
0
0
0
0


In [15]:
# Save htf
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)
    

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [16]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [17]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [18]:
test_nb_energies(htf, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -52.66819022395389), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedFo

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
test_nb_energies(htf, is_electrostatics=False, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', 13.218262478353564), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_nb_energies(htf, is_old=False, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.6703990124992907), ('HarmonicAngleForce', 8.789581233104334), ('PeriodicTorsionForce', 27.474669555944015), ('NonbondedForce', -88.46034768939744), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -86.7395212465623), ('CustomNonbondedForce', 4.748478007480066), ('CustomBondForce', -1.7208238267418343), ('CustomBondForce', 8.46978053705605), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForc

In [29]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [24]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.6703990124992907), ('HarmonicAngleForce', 8.789581233104334), ('PeriodicTorsionForce', 27.474669555944015), ('NonbondedForce', 125.47430912062403), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -135.30622121947044), ('CustomNonbondedForce', 115.76727967402559), ('CustomBondForce', 82.63803814501956), ('CustomBondForce', 9.707044201890385), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.670922707728143), ('CustomAngleForce', 9.243728658229985), ('CustomTorsionForce', 16.285403285921724), ('CustomNonbondedForce', -

# Alanine dipeptide in solvent

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
atp.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
# atp.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
atp.system.getForce(3).setUseDispersionCorrection(False)
for i in range(atp.system.getNumForces()):
    if i == 3:
        atp.system.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
    elif i < 3: # do not set PBCs for barostat
        atp.system.getForce(i).setUsesPeriodicBoundaryConditions(False)
atp.system.removeForce(4) # Remove barostat
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)
htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 16, 19, 17, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -26895.326844682648


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda'], ['nonscale_region1'], ['nonscale_region2']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, 

In [4]:
# Save htf
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

In [23]:
htf._alchemical_regions_by_type['unique_new_atoms']

{1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564}

In [24]:
htf._alchemical_regions_by_type['unique_old_atoms']

{10, 11, 12, 13}

### Test one alchemical region, no scale regions

#### Old system nb force vs hybrid system custom nb force and custom bond force (electrostatics)

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [118]:
htf._topology_proposal.old_system.getForce(3).getUseSwitchingFunction()

False

In [119]:
htf._topology_proposal.old_system.getForce(3).getNonbondedMethod()

0

In [120]:
htf._topology_proposal.old_system.getForce(3).getUseDispersionCorrection()

False

In [121]:
htf._topology_proposal.old_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [34]:
htf._topology_proposal.new_system.getForce(3).getUseSwitchingFunction()

False

In [35]:
htf._topology_proposal.new_system.getForce(3).getNonbondedMethod()

4

In [36]:
htf._topology_proposal.new_system.getForce(3).getUseDispersionCorrection()

True

In [37]:
htf._topology_proposal.new_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [40]:
htf._topology_proposal.new_system.getForce(3).getExceptionsUsePeriodicBoundaryConditions()

False

In [122]:
openmm.NonbondedForce.NoCutoff

0

In [123]:
htf.hybrid_system.getForce(3).getNonbondedMethod()

0

In [124]:
htf.hybrid_system.getForce(3).getUseSwitchingFunction()

False

In [125]:
htf.hybrid_system.getForce(3).getUseLongRangeCorrection()

False

In [126]:
htf.hybrid_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [64]:
# htf.hybrid_system.getForce(3).setCutoffDistance(1.0*unit.nanometer)

In [127]:
htf.hybrid_system.getForce(4).getNonbondedMethod()

0

In [128]:
htf.hybrid_system.getForce(4).getUseSwitchingFunction()

False

In [80]:
# htf.hybrid_system.getForce(4).setUseSwitchingFunction(False)

In [129]:
htf.hybrid_system.getForce(4).getUseSwitchingFunction()

False

In [130]:
htf.hybrid_system.getForce(4).getUseLongRangeCorrection()

False

In [131]:
htf.hybrid_system.getForce(4).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [48]:
# htf.hybrid_system.getForce(4).setCutoffDistance(1.0*unit.nanometer)

In [116]:
nb_force = htf.hybrid_system.getForce(3)
for i in range(nb_force.getNumParticles()):
    print(i, nb_force.getParticleParameters(i))

0 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.1123, 0.1123)
1 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.3662, -0.3662)
2 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.1123, 0.1123)
3 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.1123, 0.1123)
4 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5972, 0.5972)
5 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.5679, -0.5679)
6 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.4157, -0.4157)
7 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2719, 0.2719)
8 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0337, -0.0389)
9 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0823, 0.1007)
10 (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, -0.1825, -0.0)
11 (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0603, 0.0)
12 (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0603, 0.0)
13 (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0603, 0.0)
14 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5973, 0.5973)
15 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.5679, -0.5679)
16 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.4157, -0.4157)
17 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2719, 0.2719)
18 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.149, -0.149)
19 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0976, 0.0976)
20 (1.0, 

In [6]:
test_nb_energies(htf, is_solvated=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -7797.861418389372), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.08122507855787567), ('CustomAngleForce', 4.175795899528577), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7880.499360164609), ('CustomNonbondedForce', 1320.6470281163988), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', 1329.1168506768688), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.08122507855787567), ('CustomAngleForce', 4.175795899528577), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7880.499360164609), ('CustomNonbondedForce', 1320.6470281163988), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.08070138332902348), ('HarmonicAngleForce', 3.7216484744029246), ('PeriodicTorsionForce', 26.757939947410566), ('NonbondedForce', -7827.125277791792), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.08122507855787567), ('CustomAngleForce', 4.175795899528577), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7827.521717278912), ('CustomNonbondedForce', 1320.6470281163988), ('CustomBondForce', 0.3965313230069307), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.08070138332902348), ('HarmonicAngleForce', 3.7216484744029246), ('PeriodicTorsionForce', 26.757939947410566), ('NonbondedForce', 1516.6253808370082), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.08122507855787567), ('CustomAngleForce', 4.175795899528577), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7880.499360164609), ('CustomNonbondedForce', 1505.509387683281), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 11.115906995754173), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


### Test one alchemical region, one scale region

In [5]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
atp.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
atp.system.getForce(3).setUseDispersionCorrection(False)
for i in range(atp.system.getNumForces()):
    if i == 3:
        atp.system.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
    elif i < 3: # do not set PBCs for barostat
        atp.system.getForce(i).setUsesPeriodicBoundaryConditions(False)
atp.system.removeForce(4) # Remove barostat
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])


htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 17, 13, 19, 15, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -27100.597456215375
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_t

added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:Handling constraints...
INFO:relative:Handling virtual sites...
INFO:relative:	_handle_virtual_sites: numVirtualSites: 0


In [6]:
# Save htf
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [17]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [8]:
test_nb_energies(htf, is_solvated=True, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -7832.853764143414), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7915.4917055045), ('CustomNonbondedForce', 1297.3931385839335), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', 1305.8629460138732), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7915.4917055045), ('CustomNonbondedForce', 1297.3931385839335), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -791

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_nb_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.1326582532301948), ('HarmonicAngleForce', 14.321526191068509), ('PeriodicTorsionForce', 26.5452217559407), ('NonbondedForce', -7896.9236824843265), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7871.667036289536), ('CustomNonbondedForce', 1297.3931385839335), ('CustomBondForce', -25.256554215801973), ('CustomBondForce', 8.469780004620791), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedFor

In [13]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True, check_scale=True)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.1326582532301948), ('HarmonicAngleForce', 14.321526191068509), ('PeriodicTorsionForce', 26.5452217559407), ('NonbondedForce', 1377.4814027073412), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7915.4917055045), ('CustomNonbondedForce', 1354.8586052632547), ('CustomBondForce', 82.63803400070168), ('CustomBondForce', 22.622770035934536), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.133181948459047), ('CustomAngleForce', 14.77567361619416), ('CustomTorsionForce', 16.28540329352652), ('CustomNonbondedForce', -7915.

# CDK2 transformation in solvent

In [3]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [4]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

htf = setup_dict['hybrid_topology_factories']['solvent']


2021-06-29 10:14:44,002:(0.00s):root:	Detecting phases...
2021-06-29 10:14:44,005:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-06-29 10:14:44,007:(0.00s):root:No constraints will be removed
2021-06-29 10:14:44,009:(0.00s):root:No spectators
2021-06-29 10:14:44,011:(0.00s):root:			run_type is not specified; default to None
2021-06-29 10:14:44,013:(0.00s):root:	Detecting fe_type...
2021-06-29 10:14:44,015:(0.00s):root:		fe_type: repex
2021-06-29 10:14:44,017:(0.00s):root:			offline-freq not specified: default to 10.
2021-06-29 10:14:44,019:(0.00s):root:	'neglect_angles' detected: False.
2021-06-29 10:14:44,021:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-06-29 10:14:44,022:(0.00s):root:	Creating 'output'...
2021-06-29 10:14:44,028:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-06-29 10:14:44,030:(0.00s):ro

2021-06-29 10:14:47,813:(0.00s):geometry:Conducting forward proposal...
2021-06-29 10:14:47,814:(0.00s):geometry:Computing proposal order with NetworkX...
2021-06-29 10:14:47,821:(0.01s):geometry:number of atoms to be placed: 1
2021-06-29 10:14:47,822:(0.00s):geometry:Atom index proposal order is [2190]
2021-06-29 10:14:47,823:(0.00s):geometry:omitted_bonds: []
2021-06-29 10:14:47,824:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-06-29 10:14:47,948:(0.12s):geometry:creating growth system...
2021-06-29 10:14:47,982:(0.03s):geometry:	creating bond force...
2021-06-29 10:14:47,983:(0.00s):geometry:	there are 27 bonds in reference force.
2021-06-29 10:14:47,985:(0.00s):geometry:	creating angle force...
2021-06-29 10:14:47,986:(0.00s):geometry:	there are 85 angles in reference force.
2021-06-29 10:14:47,988:(0.00s):geometry:	creating torsion force...
2021-06-29 10:14:47,989:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-06-29 10:14:49,766:(1.73s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-06-29 10:15:00,503:(1.70s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 10:15:02,202:(1.70s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-06-29 10:15:02,205:(0.00s):geometry:final reduced energy 43.36216011253515
2021-06-29 10:15:02,206:(0.00s):geometry:sum of energies: 43.362159398009624
2021-06-29 10:15:02,207:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-06-29 10:15:02,208:(0.00s):geometry:Final logp_proposal: 11.562669409973775


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-06-29 10:15:02,415:(0.21s):root:	writing pickle output...
2021-06-29 10:15:02,522:(0.11s):root:	successfully dumped pickle.
2021-06-29 10:15:02,526:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-06-29 10:15:02,528:(0.00s):root:	writing atom_mapping.png
2021-06-29 10:15:03,037:(0.51s):root:	 steps per move application: 50
2021-06-29 10:15:03,039:(0.00s):root:	trajectory prefix: out
2021-06-29 10:15:03,040:(0.00s):root:	atom selection detected: not water
2021-06-29 10:15:03,041:(0.00s):root:	no nonequilibrium detected.
2021-06-29 10:15:03,042:(0.00s):root:	cataloging HybridTopologyFactories...
2021-06-29 10:15:03,044:(0.00s):root:		phase: solvent:
2021-06-29 10:15:03,045:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-06-29 10:15:03,046:(0.00s):root:		 changed nonnbondedmethod to: 0
2021-06-29 10:15:03,047:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-06-29 10:15:03,048:(0.00s):rela

In [5]:

# Set nonbonded params
# htf._topology_proposal.old_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
# htf._topology_proposal.old_system.getForce(3).setUseDispersionCorrection(False)
# htf._topology_proposal.old_system.removeForce(4) # Remove barostat

htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat

# htf.hybrid_system.removeForce(0)
# htf.hybrid_system.getForce(3).setNonbondedMethod(openmm.CustomNonbondedForce.NoCutoff)
# htf.hybrid_system.getForce(3).setUseLongRangeCorrection(False)
# htf.hybrid_system.getForce(3).setUseSwitchingFunction(False)

# htf.hybrid_system.getForce(4).setNonbondedMethod(openmm.CustomNonbondedForce.NoCutoff)
# htf.hybrid_system.getForce(4).setUseLongRangeCorrection(False)
# htf.hybrid_system.getForce(4).setUseSwitchingFunction(False)

# htf.hybrid_system.getForce(5).setUsesPeriodicBoundaryConditions(False)

# htf.hybrid_system.getForce(6).setUsesPeriodicBoundaryConditions(False)

In [6]:
# Save htf
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_nb_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', -11539.728483322744), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.549101180247508), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10682.021733558106), ('CustomNonbondedForce', 1807.0792413215015), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1830.9750656072613), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.549101180247508), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10682.021733558106), ('CustomNonbondedForce', 1807.0792413215015), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.502737994852257), ('PeriodicTorsionForce', 15.077827923252181), ('NonbondedForce', -11529.545871393288), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.549101180247508), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10693.594245750202), ('CustomNonbondedForce', 1807.0792413215015), ('CustomBondForce', -835.9514895645796), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [13]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.502737994852257), ('PeriodicTorsionForce', 15.077827923252181), ('NonbondedForce', 1834.8061120670895), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.549101180247508), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10682.021733558106), ('CustomNonbondedForce', 1810.8840952197431), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.921953028623296), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


### Test one alchemical region, one scale region

In [15]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [16]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-06-29 10:15:44,454:(40.76s):root:	Detecting phases...
2021-06-29 10:15:44,457:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-06-29 10:15:44,458:(0.00s):root:No constraints will be removed
2021-06-29 10:15:44,459:(0.00s):root:No spectators
2021-06-29 10:15:44,462:(0.00s):root:			run_type is not specified; default to None
2021-06-29 10:15:44,463:(0.00s):root:	Detecting fe_type...
2021-06-29 10:15:44,464:(0.00s):root:		fe_type: repex
2021-06-29 10:15:44,464:(0.00s):root:			offline-freq not specified: default to 10.
2021-06-29 10:15:44,465:(0.00s):root:	'neglect_angles' detected: False.
2021-06-29 10:15:44,466:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-06-29 10:15:44,466:(0.00s):root:	Creating 'output'...
2021-06-29 10:15:44,476:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-06-29 10:15:44,477:(0.00s):r

2021-06-29 10:15:48,264:(0.00s):geometry:Conducting forward proposal...
2021-06-29 10:15:48,266:(0.00s):geometry:Computing proposal order with NetworkX...
2021-06-29 10:15:48,273:(0.01s):geometry:number of atoms to be placed: 1
2021-06-29 10:15:48,275:(0.00s):geometry:Atom index proposal order is [2190]
2021-06-29 10:15:48,276:(0.00s):geometry:omitted_bonds: []
2021-06-29 10:15:48,277:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-06-29 10:15:48,579:(0.30s):geometry:creating growth system...
2021-06-29 10:15:48,616:(0.04s):geometry:	creating bond force...
2021-06-29 10:15:48,618:(0.00s):geometry:	there are 27 bonds in reference force.
2021-06-29 10:15:48,620:(0.00s):geometry:	creating angle force...
2021-06-29 10:15:48,621:(0.00s):geometry:	there are 85 angles in reference force.
2021-06-29 10:15:48,623:(0.00s):geometry:	creating torsion force...
2021-06-29 10:15:48,625:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-06-29 10:15:49,307:(0.62s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-06-29 10:15:56,359:(1.33s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 10:15:58,031:(1.67s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-06-29 10:15:58,034:(0.00s):geometry:final reduced energy 43.36216011253515
2021-06-29 10:15:58,035:(0.00s):geometry:sum of energies: 43.362159398009624
2021-06-29 10:15:58,037:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-06-29 10:15:58,039:(0.00s):geometry:Final logp_proposal: 11.821616825092365


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-06-29 10:15:58,248:(0.21s):root:	writing pickle output...
2021-06-29 10:15:58,445:(0.20s):root:	successfully dumped pickle.
2021-06-29 10:15:58,450:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-06-29 10:15:58,453:(0.00s):root:	writing atom_mapping.png
2021-06-29 10:15:58,530:(0.08s):root:	 steps per move application: 50
2021-06-29 10:15:58,533:(0.00s):root:	trajectory prefix: out
2021-06-29 10:15:58,536:(0.00s):root:	atom selection detected: not water
2021-06-29 10:15:58,538:(0.00s):root:	no nonequilibrium detected.
2021-06-29 10:15:58,540:(0.00s):root:	cataloging HybridTopologyFactories...
2021-06-29 10:15:58,541:(0.00s):root:		phase: solvent:
2021-06-29 10:15:58,543:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-06-29 10:15:58,545:(0.00s):root:		 changed nonnbondedmethod to: 0
2021-06-29 10:15:58,547:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-06-29 10:15:58,549:(0.00s):rela

In [17]:
htf = setup_dict['hybrid_topology_factories']['solvent']
htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat


In [18]:
# Save htf
with open("cdk2_solvent_scale_regions.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [21]:
test_nb_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', -11584.545697387666), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10726.838947092587), ('CustomNonbondedForce', 1758.2831577200577), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbonde

In [22]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1782.178955761827), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10726.838947092587), ('CustomNonbondedForce', 1758.2831577200577), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [24]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [25]:
test_nb_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.189496994288245), ('PeriodicTorsionForce', 15.08012310044774), ('NonbondedForce', -11576.06324935517), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10740.149129758945), ('CustomNonbondedForce', 1758.2831577200577), ('CustomBondForce', -835.9139829696472), ('CustomBondForce', 23.895760551071746), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedF

In [26]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [27]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.189496994288245), ('PeriodicTorsionForce', 15.08012310044774), ('NonbondedForce', 1786.0125918763558), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', -10726.838947092587), ('CustomNonbondedForce', 1762.0919305669918), ('CustomBondForce', -857.7066221249365), ('CustomBondForce', 23.920623734690185), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.235860179683495), ('CustomTorsionForce', 15.07675469305873), ('CustomNonbondedForce', 

# 8mer in vacuum

In [3]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [113]:
# Test 8-mer peptide in vacuum
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
# barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 1e-4,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

# modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [114]:
# set nonbonded parameters
system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
system.getForce(3).setUseDispersionCorrection(False)
for i in range(system.getNumForces()):
    if i == 3:
        system.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
    elif i < 3: # do not set PBCs for barostat
        system.getForce(i).setUsesPeriodicBoundaryConditions(False)
# system.removeForce(4) # Remove barostat


In [166]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR', 'ARG', 'LYS', 'GLU', 'ASP']]
# mutant = random.choice(aminos_updated)
mutant = "LEU"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=False)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=False)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-06-29 15:15:33,519:(9.36s):proposal_generator:	Conducting polymer point mutation proposal...
2021-06-29 15:15:33,538:(0.02s):proposal_generator:Using matching_criterion to chose best atom map
2021-06-29 15:15:33,545:(0.01s):proposal_generator:Scaffold has symmetry of 0
2021-06-29 15:15:33,547:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-06-29 15:15:33,550:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-06-29 15:15:33,555:(0.01s):proposal_generator:len [{14: 8}, {18: 9}, {17: 9}, {16: 9}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {18: 8}, {17: 8}, {16: 8}, {15: 8}, {10: 7}, {13: 8}, {12: 8}, {11: 8}, {10: 8}, {18: 7}, {17: 7}, {16: 7}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}]
2021-06-29 15:15:33,557:(0.00s):proposal_generator:{14: 8}
2021-06-29 15:15:33,559:(0.00s):proposal_generator:{18: 9}
2021-06-29 15:15:33,561:(0.00s):proposal_g

Making mutation ALA->LEU


2021-06-29 15:15:34,180:(0.51s):geometry:creating growth system...
2021-06-29 15:15:34,196:(0.02s):geometry:	creating bond force...
2021-06-29 15:15:34,198:(0.00s):geometry:	there are 64 bonds in reference force.
2021-06-29 15:15:34,201:(0.00s):geometry:	creating angle force...
2021-06-29 15:15:34,203:(0.00s):geometry:	there are 236 angles in reference force.
2021-06-29 15:15:34,208:(0.01s):geometry:	creating torsion force...
2021-06-29 15:15:34,210:(0.00s):geometry:	creating extra torsions force...
2021-06-29 15:15:34,212:(0.00s):geometry:	there are 438 torsions in reference force.
2021-06-29 15:15:34,219:(0.01s):geometry:	creating nonbonded force...
2021-06-29 15:15:34,220:(0.00s):geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
2021-06-29 15:15:34,222:(0.00s):geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
2021-06-29 15:15:34,224:(0.00s):geometry:		looping through exceptions calculating growth indices, and

conducting subsequent work with the following platform: CUDA


2021-06-29 15:15:35,044:(0.78s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 15:15:43,764:(1.67s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 15:15:45,762:(2.00s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-06-29 15:15:45,765:(0.00s):geometry:final reduced energy 187.47998570143525
2021-06-29 15:15:45,767:(0.00s):geometry:sum of energies: 187.47998611671267
2021-06-29 15:15:45,768:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-06-29 15:15:45,770:(0.00s):geometry:Final logp_proposal: -30385.015714806497


added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [167]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
scale_regions = None
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions,
                    generate_htf_for_testing=True)

2021-06-29 15:15:46,053:(0.28s):relative:*** Generating RxnHybridTopologyFactory ***
2021-06-29 15:15:46,056:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-06-29 15:15:46,082:(0.03s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-06-29 15:15:46,084:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-06-29 15:15:46,086:(0.00s):relative:No unknown forces.
2021-06-29 15:15:46,088:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 0
2021-06-29 15:15:46,089:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-06-29 15:15:46,092:(0.00s):relative:Adding and mapping new atoms to hybrid system...
2021-06-29 15:15:46,094:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
2021-06-29 15:15:46,096:(0.00s):relative:p

In [168]:

htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
# htf._topology_proposal.new_system.removeForce(4) # Remove barostat


In [169]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [119]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [120]:
test_nb_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -243.3076664147409), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.544270148118206), ('CustomAngleForce', 327.9804627059894), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -879.2155855849345), ('CustomNonbondedForce', 12.998397411394903), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [121]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [122]:
test_nb_energies(htf, is_electrostatics=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', 69.04562949047147), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.544270148118206), ('CustomAngleForce', 327.9804627059894), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -879.2155855849345), ('CustomNonbondedForce', 12.998397411394903), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
def get_params(system, is_hybrid=False):
    print(f"nb method: {system.getNonbondedMethod()}")
    if is_hybrid:
        print(f"dispersion correction: {system.getUseLongRangeCorrection()}")
    else:
        print(f"dispersion correction: {system.getUseDispersionCorrection()}")
    print(f"switching fn: {system.getUseSwitchingFunction()}")
    print(f"pbcs: {system.usesPeriodicBoundaryConditions()}")    

In [170]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)


In [171]:
test_nb_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 10.179754265626803), ('HarmonicAngleForce', 496.4186294712143), ('PeriodicTorsionForce', 174.65309353339322), ('NonbondedForce', -299.591492594999), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 10.22537889803396), ('CustomAngleForce', 496.53826211699385), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -881.7602858017719), ('CustomNonbondedForce', 12.998397411394905), ('CustomBondForce', 598.67722039197), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]


AssertionError: standard nonbonded: -299.591492594999, custom nonbonded/bond: -283.083065409802

In [145]:
get_params(htf.hybrid_system.getForce(3), is_hybrid=True)

nb method: 0
dispersion correction: False
switching fn: False
pbcs: False


In [30]:
get_params(htf.hybrid_system.getForce(4), is_hybrid=True)

nb method: 0
dispersion correction: False
switching fn: False
pbcs: False


In [56]:
htf.hybrid_system.getForce(5).usesPeriodicBoundaryConditions()

False

In [57]:
htf.hybrid_system.getForce(6).usesPeriodicBoundaryConditions()

False

In [65]:
htf._topology_proposal.new_system.getForce(3).getUseSwitchingFunction()

False

In [54]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE0-CH3
1 ACE0-C
2 ACE0-O
3 ACE0-H1
4 ACE0-H2
5 ACE0-H3
6 SER1-N
7 SER1-CA
8 SER1-C
9 SER1-O
10 SER1-CB
11 SER1-OG
12 SER1-H2
13 SER1-HA
14 SER1-HB3
15 SER1-HB2
16 SER1-HG
17 ALA2-N
18 ALA2-CA
19 ALA2-C
20 ALA2-O
21 ALA2-CB
22 ALA2-H
23 ALA2-HA
24 ALA2-HB1
25 ALA2-HB2
26 ALA2-HB3
123 ALA2-CD1
124 ALA2-HD11
125 ALA2-HD12
126 ALA2-HD13
127 ALA2-CB
128 ALA2-HB
129 ALA2-CG2
130 ALA2-HG21
131 ALA2-HG22
132 ALA2-HG23
133 ALA2-CG1
134 ALA2-HG12
135 ALA2-HG13
27 VAL3-N
28 VAL3-CA
29 VAL3-C
30 VAL3-O
31 VAL3-CB
32 VAL3-CG1
33 VAL3-CG2
34 VAL3-H
35 VAL3-HA
36 VAL3-HB
37 VAL3-HG11
38 VAL3-HG12
39 VAL3-HG13
40 VAL3-HG21
41 VAL3-HG22
42 VAL3-HG23
43 LEU4-N
44 LEU4-CA
45 LEU4-C
46 LEU4-O
47 LEU4-CB
48 LEU4-CG
49 LEU4-CD1
50 LEU4-CD2
51 LEU4-H
52 LEU4-HA
53 LEU4-HB3
54 LEU4-HB2
55 LEU4-HG
56 LEU4-HD11
57 LEU4-HD12
58 LEU4-HD13
59 LEU4-HD21
60 LEU4-HD22
61 LEU4-HD23
62 GLN5-N
63 GLN5-CA
64 GLN5-C
65 GLN5-O
66 GLN5-CB
67 GLN5-CG
68 GLN5-CD
69 GLN5-NE2
70 GLN5-OE1
71 GLN5-H
72 GLN5-HA
73 GLN5-HB3
74 

In [55]:
nb_force = htf.hybrid_system.getForce(3)
for i in range(nb_force.getNumParticles()):
    print(i, nb_force.getParticleParameters(i))

0 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.3662, -0.0)
1 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5972, 0.0)
2 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.5679, -0.0)
3 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1123, 0.0)
4 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1123, 0.0)
5 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1123, 0.0)
6 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.4157, -0.4157)
7 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.0249, -0.0249)
8 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5973, 0.5973)
9 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.5679, -0.5679)
10 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2117, 0.2117)
11 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.6546, -0.6546)
12 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2719, 0.2719)
13 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0843, 0.0843)
14 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0352, 0.0352)
15 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0352, 0.0352)
16 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.4275, 0.4275)
17 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.4157, -0.4157)
18 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0337, -0.0597)
19 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5973, 0.5973)
20 (1.0, 0.0,

In [48]:
htf.hybrid_system.getForce(3).getEnergyFunction()

'U_electrostatics; U_electrostatics = switch * c_RF * u_RF; switch = 1; t = (r_eff - r_switch) / (r_cutoff - r_switch); c_RF = 138.935456 * chargeProd; chargeProd = charge1 * charge2; charge1 = ((charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 )) + (charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ))) * ( nonscale_lambda_electrostatics * nonscale_region1 ); charge2 = ((charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 )) + (charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ))) * ( nonscale_lambda_electrostatics * nonscale_region2 ); u_RF = 1 / r_eff; r_eff = sqrt(r^2 + w^2); w = step(unique_new1 + unique_new2 + unique_old1 + unique_old2 - 0.1) * r_cutoff * (1. - ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 ) - ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 )) *

In [ ]:
U_electrostatics;
U_electrostatics = switch * c_RF * u_RF;
switch = 1;
t = (r_eff - r_switch) / (r_cutoff - r_switch);
c_RF = 138.935456 * chargeProd;
chargeProd = charge1 * charge2;
charge1 = ((charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 )) + (charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ))) * ( nonscale_lambda_electrostatics * nonscale_region1 );
charge2 = ((charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 )) + (charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ))) * ( nonscale_lambda_electrostatics * nonscale_region2 );
u_RF = 1 / r_eff;
r_eff = sqrt(r^2 + w^2);
w = step(unique_new1 + unique_new2 + unique_old1 + unique_old2 - 0.1) * r_cutoff * (1. - ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 ) - ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 )) * 0;
r_switch = r_cutoff * 0.25;
eps_RF = 78.8;
r_cutoff = 1.4;
w_scale = 0;'

In [152]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)


In [153]:
test_nb_energies(htf, is_electrostatics=False, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 9.123179895657778), ('HarmonicAngleForce', 346.0420650607845), ('PeriodicTorsionForce', 164.39865377562708), ('NonbondedForce', 951.6044721926664), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 9.168804528064936), ('CustomAngleForce', 346.1616977065642), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -879.2155855849345), ('CustomNonbondedForce', 732.2363821130278), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 219.36802668850493), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


# 8mer in solvent

In [74]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [75]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 1e-4,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [76]:
# set nonbonded parameters
system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
# atp.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
system.getForce(3).setUseDispersionCorrection(False)
for i in range(system.getNumForces()):
    if i == 3:
        system.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
    elif i < 3: # do not set PBCs for barostat
        system.getForce(i).setUsesPeriodicBoundaryConditions(False)
system.removeForce(4) # Remove barostat


In [77]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
# mutant = "MET"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-06-29 15:04:45,571:(240.13s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->CYS


2021-06-29 15:04:46,094:(0.52s):proposal_generator:Using matching_criterion to chose best atom map
2021-06-29 15:04:46,099:(0.00s):proposal_generator:Scaffold has symmetry of 0
2021-06-29 15:04:46,100:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-06-29 15:04:46,102:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-06-29 15:04:46,105:(0.00s):proposal_generator:len [{8: 7}, {9: 7}, {10: 7}, {8: 8}, {9: 8}, {10: 8}, {8: 9}, {9: 9}, {10: 9}]
2021-06-29 15:04:46,107:(0.00s):proposal_generator:{8: 7}
2021-06-29 15:04:46,108:(0.00s):proposal_generator:{9: 7}
2021-06-29 15:04:46,110:(0.00s):proposal_generator:{10: 7}
2021-06-29 15:04:46,112:(0.00s):proposal_generator:{8: 8}
2021-06-29 15:04:46,114:(0.00s):proposal_generator:{9: 8}
2021-06-29 15:04:46,115:(0.00s):proposal_generator:{10: 8}
2021-06-29 15:04:46,117:(0.00s):proposal_generator:{8: 9}
2021-06-29 15:04:4

conducting subsequent work with the following platform: CUDA


2021-06-29 15:04:50,383:(2.32s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 15:05:05,752:(2.49s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 15:05:08,421:(2.67s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-06-29 15:05:08,422:(0.00s):geometry:final reduced energy 187.47998570143525
2021-06-29 15:05:08,423:(0.00s):geometry:sum of energies: 187.47998611671267
2021-06-29 15:05:08,423:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-06-29 15:05:08,424:(0.00s):geometry:Final logp_proposal: -30393.95753461393


added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [78]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = None
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions,
                    generate_htf_for_testing=True)

2021-06-29 15:05:08,693:(0.27s):relative:*** Generating RxnHybridTopologyFactory ***
2021-06-29 15:05:08,694:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-06-29 15:05:08,842:(0.15s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-06-29 15:05:08,844:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-06-29 15:05:08,846:(0.00s):relative:No unknown forces.
2021-06-29 15:05:08,847:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 0
2021-06-29 15:05:08,849:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-06-29 15:05:08,895:(0.05s):relative:Adding and mapping new atoms to hybrid system...
2021-06-29 15:05:08,897:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
2021-06-29 15:05:08,

In [79]:

htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat


In [80]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [81]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [82]:
test_nb_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -28626.686752216232), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.491148170545303), ('CustomAngleForce', 230.00398956217666), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29262.59433545261), ('CustomNonbondedForce', 4513.45654351265), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [83]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [84]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', 4569.503982925087), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.491148170545303), ('CustomAngleForce', 230.00398956217666), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29262.59433545261), ('CustomNonbondedForce', 4513.45654351265), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [85]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [86]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.445523538138145), ('HarmonicAngleForce', 229.88435691639694), ('PeriodicTorsionForce', 163.39828800693036), ('NonbondedForce', -28578.34070560514), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.491148170545303), ('CustomAngleForce', 230.00398956217666), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29214.24475358768), ('CustomNonbondedForce', 4513.45654351265), ('CustomBondForce', 635.9043718488828), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [87]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [88]:
test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.445523538138145), ('HarmonicAngleForce', 229.88435691639694), ('PeriodicTorsionForce', 163.39828800693036), ('NonbondedForce', 33209.110127626715), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.491148170545303), ('CustomAngleForce', 230.00398956217666), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29262.59433545261), ('CustomNonbondedForce', 32959.76599486748), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 249.34408557416174), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


### Test one alchemical region, one scale region

In [89]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [198]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 1e-4,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [199]:
# set nonbonded parameters
system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
system.getForce(3).setUseDispersionCorrection(False)
for i in range(system.getNumForces()):
    if i == 3:
        system.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
    elif i < 3: # do not set PBCs for barostat
        system.getForce(i).setUsesPeriodicBoundaryConditions(False)
system.removeForce(4) # Remove barostat


In [200]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
# mutant = random.choice(aminos_updated)
mutant = "ASN"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-06-29 17:13:34,498:(117.93s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->ASN


2021-06-29 17:13:34,886:(0.39s):proposal_generator:Using matching_criterion to chose best atom map
2021-06-29 17:13:34,890:(0.00s):proposal_generator:Scaffold has symmetry of 0
2021-06-29 17:13:34,892:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-06-29 17:13:34,894:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-06-29 17:13:34,897:(0.00s):proposal_generator:len [{10: 7}, {11: 7}, {12: 7}, {13: 7}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
2021-06-29 17:13:34,899:(0.00s):proposal_generator:{10: 7}
2021-06-29 17:13:34,901:(0.00s):proposal_generator:{11: 7}
2021-06-29 17:13:34,903:(0.00s):proposal_generator:{12: 7}
2021-06-29 17:13:34,905:(0.00s):proposal_generator:{13: 7}
2021-06-29 17:13:34,907:(0.00s):proposal_generator:{10: 8}
2021-06-29 17:13:34,909:(0.00s):proposal_generator:{11: 8}
2021-06-29 17:13:34,910:(0.00s):proposa

conducting subsequent work with the following platform: CUDA


2021-06-29 17:13:38,227:(1.35s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 17:13:49,428:(1.41s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-06-29 17:13:51,167:(1.74s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-06-29 17:13:51,169:(0.00s):geometry:final reduced energy 187.47998570143525
2021-06-29 17:13:51,171:(0.00s):geometry:sum of energies: 187.47998611671267
2021-06-29 17:13:51,173:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-06-29 17:13:51,174:(0.00s):geometry:Final logp_proposal: -30541.714864566547


added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [201]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = [list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])]
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions,
                    generate_htf_for_testing=True)

2021-06-29 17:13:51,441:(0.27s):relative:*** Generating RxnHybridTopologyFactory ***
2021-06-29 17:13:51,444:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-06-29 17:13:51,579:(0.13s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-06-29 17:13:51,581:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-06-29 17:13:51,582:(0.00s):relative:No unknown forces.
2021-06-29 17:13:51,584:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 0
2021-06-29 17:13:51,585:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-06-29 17:13:51,630:(0.04s):relative:Adding and mapping new atoms to hybrid system...
2021-06-29 17:13:51,631:(0.00s):relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], 

In [202]:

htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
htf._topology_proposal.new_system.removeForce(4) # Remove barostat


In [203]:
# Save htf
with open("8mer_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [204]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [205]:
test_nb_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -28612.26277820097), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29248.170361608074), ('CustomNonbondedForce', 4420.23487505275), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce'

In [206]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [207]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', 4476.282260838404), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29248.170361608074), ('CustomNonbondedForce', 4420.23487505275), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29248

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [208]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [209]:
test_nb_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 12.952238370325562), ('HarmonicAngleForce', 309.5078158922283), ('PeriodicTorsionForce', 178.23722098859298), ('NonbondedForce', -28733.951113703442), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29263.70683227857), ('CustomNonbondedForce', 4420.23487505275), ('CustomBondForce', 529.7560526664622), ('CustomBondForce', 56.047220181697135), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce

In [210]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [211]:
test_nb_energies(htf, is_old=False, is_electrostatics=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 12.952238370325562), ('HarmonicAngleForce', 309.5078158922283), ('PeriodicTorsionForce', 178.23722098859298), ('NonbondedForce', 7758.398542253916), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29248.170361608074), ('CustomNonbondedForce', 7559.23950269198), ('CustomBondForce', 635.907909782493), ('CustomBondForce', 199.15714640775045), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.99786300273272), ('CustomAngleForce', 309.627448538008), ('CustomTorsionForce', 156.24351836067592), ('CustomNonbondedForce', -29248

## barnase:barstar (no naked charge fix, no counterion fix)

### Test one alchemical region, no scale regions

In [4]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor

In [20]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol, oechem
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RxnHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 repartitioned_endstate=None,
                 rxn_field = True,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            repartitioned_endstate : int, default None
                the endstate (0 or 1) at which to build the RepartitionedHybridTopologyFactory. By default, this is None,
                meaning a vanilla HybridTopologyFactory will be built.
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for (top, pos, sys) in inputs:
            
            sys.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
            sys.getForce(3).setUseDispersionCorrection(False)
            for i in range(sys.getNumForces()):
                if i == 3:
                    sys.getForce(i).setExceptionsUsePeriodicBoundaryConditions(False)
                elif i < 3: # do not set PBCs for barostat
                    sys.getForce(i).setUsesPeriodicBoundaryConditions(False)
            sys.removeForce(4) # Remove barostat

            
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            if rxn_field:
                factory = RxnHybridTopologyFactory
            else:
                if repartitioned_endstate is None:
                    factory = HybridTopologyFactory
                elif repartitioned_endstate in [0, 1]:
                    factory = RepartitionedHybridTopologyFactory

            forward_htf = factory(topology_proposal=topology_proposal,
                                  current_positions=pos,
                                  new_positions=new_positions,
                                  use_dispersion_correction=False,
                                  functions=None,
                                  softcore_alpha=None,
                                  bond_softening_constant=1.0,
                                  angle_softening_constant=1.0,
                                  soften_only_new=False,
                                  neglected_new_angle_terms=[],
                                  neglected_old_angle_terms=[],
                                  softcore_LJ_v2=True,
                                  softcore_electrostatics=True,
                                  softcore_LJ_v2_alpha=0.85,
                                  softcore_electrostatics_alpha=0.3,
                                  softcore_sigma_Q=1.0,
                                  interpolate_old_and_new_14s=flatten_exceptions,
                                  omitted_terms=None,
                                  endstate=repartitioned_endstate,
                                  flatten_torsions=flatten_torsions,
                                 generate_htf_for_testing=True)

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass

            htfs.append(forward_htf)

        self.apo_htf = htfs[0]
        self.complex_htf = htfs[1] if ligand_input else None

In [21]:
solvent_delivery = PointMutationExecutor2("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        rxn_field=True
                       )

INFO:root:solvating at 0.05 M using tip3p
INFO:root:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:pr

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.05560346848027731.
INFO:geometry:	reduced angle potential = 0.015530549903522818.
INFO:geometry:	reduced angle potential = 1.5852325185944984.
INFO:geometry:	reduced angle potential = 1.130309680909011.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4939 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -62.39009687497522
INFO:geometry:final reduced energy 4730.077774354767
INFO:geometry:sum of energies: 4730.077773802322
INFO:geometry:magnitude of difference in the energies: 5.524455772842884e-07
INFO:geometry:Final logp_proposal: -9679.161065999691


added energy components: [('CustomBondForce', 0.4083956553523612), ('CustomAngleForce', 1.622079514297996), ('CustomTorsionForce', 17.50313698245956), ('CustomBondForce', -81.92370902708515)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:particle (nonbonded) scale_templates: [['nonscale_lambda'], ['nonscale_region1'], ['nonscale_region2']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.

INFO:relative:hybrid_index_pair (2588, 2592, 2590, 2600) was not found in the new_term_collector, but (2592, 2588, 2590, 2600) has the same atoms and terms, so (2592, 2588, 2590, 2600) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2644, 2641, 2642, 2651) was not found in the new_term_collector, but (2641, 2644, 2642, 2651) has the same atoms and terms, so (2641, 2644, 2642, 2651) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2663, 2665, 2664, 2666) was not found in the new_term_collector, but (2665, 2664, 2663, 2666) has the same atoms and terms, so (2665, 2664, 2663, 2666) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2787, 2784, 2785, 2795) was not found in the new_term_collector, but (2784, 2787, 2785, 2795) has the same atoms and terms, so (2784, 2787, 2785, 2795) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2788, 2784, 2786, 2796) was not found in the new_te

In [23]:
complex_htf = solvent_delivery.get_complex_htf()

complex_htf._topology_proposal.new_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
complex_htf._topology_proposal.new_system.getForce(3).setUseDispersionCorrection(False)
complex_htf._topology_proposal.new_system.removeForce(4) # Remove barostat



In [27]:
# pickle.dump(apo_htf, open("T42A_apo.pickle", "wb" ))
pickle.dump(complex_htf, open("T42A_complex.pickle", "wb" ))


#### Old system PeroidicTorsionForce vs hybrid system CustomTorsionForce

In [28]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_nb_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', -156915.80021943303), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.1014990334312), ('CustomAngleForce', 988.9471353798067), ('CustomTorsionForce', 3830.526609908918), ('CustomNonbondedForce', -171276.3928885331), ('CustomNonbondedForce', 56941.32190437939), ('CustomBondForce', 14360.59463495793), ('CustomBondForce', 1038.0966743330835), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [30]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [31]:
test_nb_energies(htf, is_electrostatics=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', 57979.4165972208), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.1014990334312), ('CustomAngleForce', 988.9471353798067), ('CustomTorsionForce', 3830.526609908918), ('CustomNonbondedForce', -171276.3928885331), ('CustomNonbondedForce', 56941.32190437939), ('CustomBondForce', 14360.59463495793), ('CustomBondForce', 1038.0966743330835), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [32]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.69310337807886), ('HarmonicAngleForce', 987.3250558655088), ('PeriodicTorsionForce', 3824.333851956954), ('NonbondedForce', -156840.5046198266), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.1014990334312), ('CustomAngleForce', 988.9471353798067), ('CustomTorsionForce', 3830.526609908918), ('CustomNonbondedForce', -171297.47395235125), ('CustomNonbondedForce', 56941.32190437939), ('CustomBondForce', 14456.971298751589), ('CustomBondForce', 1038.0966743330835), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force electrostatics energies are equal!


In [3]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
context = test_nb_energies(htf, is_electrostatics=False, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.69310337808287), ('HarmonicAngleForce', 987.3250558655093), ('PeriodicTorsionForce', 3824.333851956955), ('NonbondedForce', 58507.19817341124), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.1014990334312), ('CustomAngleForce', 988.9471353798067), ('CustomTorsionForce', 3830.526609908918), ('CustomNonbondedForce', -171276.3928885331), ('CustomNonbondedForce', 56956.17243688812), ('CustomBondForce', 14360.59463495793), ('CustomBondForce', 1551.0277152664694), ('AndersenThermostat', 0.0)]
Success! Custom nb force and standard nb force sterics energies are equal!


In [5]:
for param, val in context.getParameters().items():
    print(param, val)

AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
lambda_0_angles_new 0.0
lambda_0_angles_old 1.0
lambda_0_bonds_new 0.0
lambda_0_bonds_old 1.0
lambda_0_electrostatics_exceptions_new 0.0
lambda_0_electrostatics_exceptions_old 1.0
lambda_0_electrostatics_new 0.0
lambda_0_electrostatics_old 1.0
lambda_0_sterics_exceptions_new 1.0
lambda_0_sterics_exceptions_old 0.0
lambda_0_sterics_new 1.0
lambda_0_sterics_old 0.0
lambda_0_torsions_new 0.0
lambda_0_torsions_old 1.0
nonscale_lambda_angles 1.0
nonscale_lambda_bonds 1.0
nonscale_lambda_electrostatics 1.0
nonscale_lambda_electrostatics_exceptions 1.0
nonscale_lambda_sterics 1.0
nonscale_lambda_sterics_exceptions 1.0
nonscale_lambda_torsions 1.0


In [ ]:
# TO DO: actually test the scaling of the energies matches